In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from utils import *
import pandas as pd

data = np.load('data.npz')
train_data = data['train_data']
test_data = data['test_data']
train_labels = data['train_labels']

train_data.shape, train_labels.shape, test_data.shape

In [ ]:
train_data = tf.cast(train_data, tf.float32).numpy()
train_labels = tf.one_hot(train_labels, 20).numpy()

In [ ]:
def se_module(seq, reduction_ratio = 16):
    #[None, time_steps, channels]
    se = keras.layers.GlobalAveragePooling1D()(seq)
    #[None, channels]
    se = keras.layers.Dense(seq.shape[-1] / reduction_ratio, activation='relu')(se)
    se = keras.layers.Dense(seq.shape[-1], activation='relu')(se)
    se = tf.nn.sigmoid(se)
    #[None, channels]
    se = keras.layers.Reshape([1, seq.shape[-1]])(se)
    #[None, 1, channels]
    return tf.multiply(seq, se)

In [ ]:
def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape[1:])
    masking = keras.layers.Masking(mask_value=0.0)(inputs)
    seq = keras.layers.LSTM(128, return_sequences=True,
                                unit_forget_bias=False,
                                recurrent_regularizer=keras.regularizers.l2(0.03),
    )(masking)
    seq = keras.layers.LayerNormalization()(seq)
    seq = se_module(seq)
    forward_layer = keras.layers.LSTM(128, return_sequences=True,
                        unit_forget_bias=False,
                        recurrent_regularizer = keras.regularizers.l2(0.015),
                        )
    backward_layer = keras.layers.LSTM(128, return_sequences=True,
                        go_backwards=True,
                        unit_forget_bias=False,
                        recurrent_regularizer = keras.regularizers.l2(0.005),
                        )
    seq = keras.layers.Bidirectional(
                        layer=forward_layer,
                        backward_layer=backward_layer,
                        )(seq)
    seq = keras.layers.GlobalAveragePooling1D()(seq)
    seq = keras.layers.Dense(512, activation='relu')(seq)
    seq = keras.layers.Dropout(0.3)(seq)
    seq = keras.layers.Dense(128, activation='relu')(seq)
    seq = keras.layers.Dropout(0.3)(seq)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(seq)

    model = keras.models.Model(inputs=[inputs], outputs=[outputs])

    model.compile(optimizer=tf.optimizers.Adam(),
            loss=tf.losses.CategoricalCrossentropy(label_smoothing=0.1),           
            metrics=['accuracy'])
    return model

In [ ]:
def scheduler_func(epoch):
    if epoch < 15:
        rate = 1e-3
    elif epoch >= 15 and epoch < 25:
        rate = 3e-4
    elif epoch >= 25:
        rate = 9e-5
    return rate * 1.0

In [ ]:
kfcv_fit(builder=lambda : build_model(train_data.shape, 20),
                x=train_data, y=train_labels,
                epochs=40,
                checkpoint_path = './models/lstm/',
                batch_size=64,
                extra_callbacks=[keras.callbacks.LearningRateScheduler(scheduler_func)],
                verbose=2,
                noise_std=0.05
                )

In [ ]:
np.save('lstm_pred.npy', kfcv_predict('models/lstm', test_data))